In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten, Embedding
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [6]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_lg')

In [7]:
word2vec_embedding_len = len(nlp.vocab['apple'].vector)
print("EMBEDDINGS_LEN=", word2vec_embedding_len )  # 300

EMBEDDINGS_LEN= 300


In [8]:
df = pd.read_csv(os.path.join(os.getcwd(),'sarcasm_headline_dataset.csv'))
df.head()

,HEADLINE,IS_SARCASTIC
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [9]:
data = df['HEADLINE'].values.tolist()
labels = df['IS_SARCASTIC'].values.tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

In [10]:
word_index = tokenizer.word_index
max_words_feature_space = len(word_index) + 1
max_seq_len = max([len(headline) for headline in data])

In [11]:
X = pad_sequences(sequences, maxlen=max_seq_len)
y = np.asarray(labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20)

In [14]:
my_embedding_matrix = np.zeros((max_words_feature_space, word2vec_embedding_len))
for word, idx in word_index.items():
    try:
        embedding = nlp.vocab[word].vector
        my_embedding_matrix[idx] = embedding
    except:
        pass

In [34]:
model1_filename = os.path.join(os.getcwd(),'models','sarcasm','word2vec_model2_1.h5')

In [21]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping

plateau_callback =ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2)
modelcheckpoint_callback = ModelCheckpoint(filepath=model1_filename,monitor='val_loss',save_best_only=True)
earlystop_callback = EarlyStopping(monitor='val_loss',patience=3)

In [22]:
model1 = Sequential()
model1.add(Embedding(max_words_feature_space,
                     word2vec_embedding_len,
                     weights=[my_embedding_matrix],
                     input_length=max_seq_len,
                     trainable=False))
model1.add(LSTM(300,return_sequences=False))
model1.add(Dense(units=1, activation='sigmoid'))

model1.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
print(model1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 254, 300)          8897100   
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total params: 9,618,601
Trainable params: 721,501
Non-trainable params: 8,897,100
_________________________________________________________________
None


In [23]:
history1 = model1.fit(X_train, y_train, 
                      epochs=30, batch_size=256,
                      verbose=1,callbacks=[plateau_callback,modelcheckpoint_callback,earlystop_callback],
                      validation_data=(X_val, y_val)
                     )

Train on 17093 samples, validate on 4274 samples
Epoch 1/30
17093/17093 [==============================] - 253s - loss: 0.6542 - acc: 0.6680 - val_loss: 0.5847 - val_acc: 0.7022
Epoch 2/30
17093/17093 [==============================] - 251s - loss: 0.4959 - acc: 0.7756 - val_loss: 0.3872 - val_acc: 0.8250
Epoch 3/30
17093/17093 [==============================] - 251s - loss: 0.4331 - acc: 0.8034 - val_loss: 0.3926 - val_acc: 0.8226
Epoch 4/30
17093/17093 [==============================] - 251s - loss: 0.3810 - acc: 0.8312 - val_loss: 0.3669 - val_acc: 0.8329
Epoch 5/30
17093/17093 [==============================] - 252s - loss: 0.3217 - acc: 0.8602 - val_loss: 0.3298 - val_acc: 0.8512
Epoch 6/30
17093/17093 [==============================] - 253s - loss: 0.2839 - acc: 0.8779 - val_loss: 0.3104 - val_acc: 0.8652
Epoch 7/30
17093/17093 [==============================] - 253s - loss: 0.2804 - acc: 0.8883 - val_loss: 0.3196 - val_acc: 0.8617
Epoch 8/30
17093/17093 [========================

In [25]:
model1.evaluate(X_test, y_test)

5342/5342 [==============================] - 46s    


[0.32871628757140825, 0.8732684388092866]

In [26]:
model1_5 = Sequential()
model1_5.add(Embedding(max_words_feature_space,
                     word2vec_embedding_len,
                     weights=[my_embedding_matrix],
                     input_length=max_seq_len,
                     trainable=False))
model1_5.add(LSTM(300,return_sequences=False))
model1_5.add(Dense(units=150, activation='relu'))
model1_5.add(Dense(units=1, activation='sigmoid'))

model1_5.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
print(model1_5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 254, 300)          8897100   
_________________________________________________________________
lstm_3 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_3 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 151       
Total params: 9,663,601
Trainable params: 766,501
Non-trainable params: 8,897,100
_________________________________________________________________
None


In [28]:
history1_5 = model1_5.fit(X_train, y_train, 
                      epochs=30, batch_size=256,
                      verbose=1,callbacks=[plateau_callback,modelcheckpoint_callback,earlystop_callback],
                      validation_data=(X_val, y_val)
                     )

Train on 17093 samples, validate on 4274 samples
Epoch 1/30
17093/17093 [==============================] - 254s - loss: 0.6049 - acc: 0.7150 - val_loss: 0.6490 - val_acc: 0.5849
Epoch 2/30
17093/17093 [==============================] - 252s - loss: 0.6046 - acc: 0.6835 - val_loss: 0.6088 - val_acc: 0.6549
Epoch 3/30
17093/17093 [==============================] - 253s - loss: 0.4971 - acc: 0.7858 - val_loss: 0.3941 - val_acc: 0.8205
Epoch 4/30
17093/17093 [==============================] - 252s - loss: 0.4996 - acc: 0.7694 - val_loss: 0.3715 - val_acc: 0.8379
Epoch 5/30
17093/17093 [==============================] - 253s - loss: 0.3284 - acc: 0.8563 - val_loss: 0.3669 - val_acc: 0.8421
Epoch 6/30
17093/17093 [==============================] - 253s - loss: 0.2865 - acc: 0.8759 - val_loss: 0.3197 - val_acc: 0.8577
Epoch 7/30
17093/17093 [==============================] - 253s - loss: 0.2464 - acc: 0.8969 - val_loss: 0.3394 - val_acc: 0.8631
Epoch 8/30
17093/17093 [========================

In [29]:
model1_5.evaluate(X_test, y_test)

5342/5342 [==============================] - 47s    


[0.368838570523869, 0.8685885434604146]

In [31]:
model2 = Sequential()
model2.add(Embedding(max_words_feature_space,
                     word2vec_embedding_len,
                     weights=[my_embedding_matrix],
                     input_length=max_seq_len,
                     trainable=False))
model2.add(LSTM(300,return_sequences=True))
model2.add(LSTM(300,return_sequences=False))
model2.add(Dense(units=1, activation='sigmoid'))

model2.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 254, 300)          8897100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 254, 300)          721200    
_________________________________________________________________
lstm_5 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 301       
Total params: 10,339,801
Trainable params: 1,442,701
Non-trainable params: 8,897,100
_________________________________________________________________
None


In [32]:
history2 = model2.fit(X_train, y_train, 
                      epochs=30, batch_size=512,
                      verbose=1,callbacks=[plateau_callback,modelcheckpoint_callback,earlystop_callback],
                      validation_data=(X_val, y_val)
                     )

Train on 17093 samples, validate on 4274 samples
Epoch 1/30
17093/17093 [==============================] - 508s - loss: 0.5740 - acc: 0.7087 - val_loss: 0.4368 - val_acc: 0.8018
Epoch 2/30
17093/17093 [==============================] - 505s - loss: 0.4217 - acc: 0.8081 - val_loss: 0.4178 - val_acc: 0.8133
Epoch 3/30
17093/17093 [==============================] - 507s - loss: 0.3804 - acc: 0.8299 - val_loss: 0.3830 - val_acc: 0.8336
Epoch 4/30
17093/17093 [==============================] - 504s - loss: 0.3449 - acc: 0.8473 - val_loss: 0.9715 - val_acc: 0.5082
Epoch 5/30
17093/17093 [==============================] - 503s - loss: 0.3346 - acc: 0.8547 - val_loss: 0.3559 - val_acc: 0.8493
Epoch 6/30
17093/17093 [==============================] - 499s - loss: 0.2837 - acc: 0.8801 - val_loss: 0.3309 - val_acc: 0.8584
Epoch 7/30
17093/17093 [==============================] - 497s - loss: 0.2576 - acc: 0.8896 - val_loss: 0.3530 - val_acc: 0.8482
Epoch 8/30
17093/17093 [========================

In [33]:
from keras.layers import Dropout

In [35]:
model2_1 = Sequential()
model2_1.add(Embedding(max_words_feature_space,
                     word2vec_embedding_len,
                     weights=[my_embedding_matrix],
                     input_length=max_seq_len,
                     trainable=False))
model2_1.add(LSTM(300,return_sequences=True))
model2_1.add(Dropout(0.2))
model2_1.add(LSTM(300,return_sequences=False))
model2_1.add(Dense(units=1, activation='sigmoid'))

model2_1.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])
print(model2_1.summary())

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 254, 300)          8897100   
_________________________________________________________________
lstm_6 (LSTM)                (None, 254, 300)          721200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 254, 300)          0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 301       
Total params: 10,339,801
Trainable params: 1,442,701
Non-trainable params: 8,897,100
_________________________________________________________________
None


In [36]:
history2_1 = model2_1.fit(X_train, y_train, 
                      epochs=30, batch_size=512,
                      verbose=1,callbacks=[plateau_callback,modelcheckpoint_callback,earlystop_callback],
                      validation_data=(X_val, y_val)
                     )

Train on 17093 samples, validate on 4274 samples
Epoch 1/30
17093/17093 [==============================] - 510s - loss: 0.5712 - acc: 0.7033 - val_loss: 0.4359 - val_acc: 0.7934
Epoch 2/30
17093/17093 [==============================] - 4822s - loss: 0.4961 - acc: 0.7872 - val_loss: 0.4182 - val_acc: 0.8156
Epoch 3/30
17093/17093 [==============================] - 501s - loss: 0.3867 - acc: 0.8292 - val_loss: 0.4003 - val_acc: 0.8208
Epoch 4/30
17093/17093 [==============================] - 499s - loss: 0.3729 - acc: 0.8339 - val_loss: 0.3753 - val_acc: 0.8339
Epoch 5/30
17093/17093 [==============================] - 613s - loss: 0.3267 - acc: 0.8598 - val_loss: 0.4268 - val_acc: 0.8084
Epoch 6/30
17093/17093 [==============================] - 624s - loss: 0.3218 - acc: 0.8594 - val_loss: 0.3296 - val_acc: 0.8538
Epoch 7/30
17093/17093 [==============================] - 583s - loss: 0.2967 - acc: 0.8713 - val_loss: 0.3150 - val_acc: 0.8627
Epoch 8/30
17093/17093 [=======================

In [37]:
model2_1.evaluate(X_test, y_test)

5342/5342 [==============================] - 94s    


[0.35054774643753583, 0.8715836765479613]